In [1]:
# імпорт бібліотек

from transformers import BertTokenizer, BertForSequenceClassification
import torch

import pandas as pd


### Prepare grouped data

In [2]:
# завантаження даних
df_news = pd.read_csv(r'C:\Users\adima\Desktop\thesis\data_overview\data\pre_ready_data\GSnewsFT.csv')

In [3]:
# кількість пропущених даних
df_news.isna().sum()

Date          0
Heading       0
Standfirst    0
dtype: int64

In [4]:
df_news

,Date,Heading,Standfirst
0,2010-02-25,Bright future,"...An investment from Goldman Sachs, the US ba..."
1,2010-02-25,Decision makers,"...He sits on the board of Goldman Sachs, is a..."
2,2010-02-25,Financial crisis panel to recall bank chiefs,...Sachs had been “creating and selling securi...
3,2010-02-25,Dimon days,...Sachs and Morgan Stanley....
4,2010-02-25,US stocks fall on Greece debt fears,"...Nike, the sports wear company, rose 1.9 per..."
...,...,...,...
15931,2021-04-29,"Margin debt and leverage are flashing red, again","...As John Waldron, chief operating officer of..."
15932,2021-04-29,Goldman Sachs predicts quantum computing 5 yea...,...Sachs....
15933,2021-04-29,No media on Verizon’s horizon,...Sachs....
15934,2021-04-30,A new model of ‘trickle down sustainability’ ...,"...On Thursday, investors sent a strong messag..."


#### Clean text

In [5]:
df_proc = df_news.copy()

In [6]:
cols = ['Heading', 'Standfirst']

In [7]:
# очищення даних

for col in cols:
    df_proc[col].replace('[^\w\s]', '', regex=True, inplace=True) # видалення всіх символів окрім букв та чисел
    df_proc[col] = df_proc[col].str.lower() # переведення в нижній регістр
    df_proc[col] = df_proc[col].str.split() # видалення пробілів
    df_proc['Standfirst'][df_proc['Standfirst'].str.len() == 1] = ' ' # видалення одиничних слів
    
    df_proc[col] = df_proc[col].str.join(' ')
df_proc

,Date,Heading,Standfirst
0,2010-02-25,bright future,an investment from goldman sachs the us bank h...
1,2010-02-25,decision makers,he sits on the board of goldman sachs is a mem...
2,2010-02-25,financial crisis panel to recall bank chiefs,sachs had been creating and selling securities...
3,2010-02-25,dimon days,sachs and morgan stanley
4,2010-02-25,us stocks fall on greece debt fears,nike the sports wear company rose 19 per cent ...
...,...,...,...
15931,2021-04-29,margin debt and leverage are flashing red again,as john waldron chief operating officer of gol...
15932,2021-04-29,goldman sachs predicts quantum computing 5 yea...,
15933,2021-04-29,no media on verizons horizon,
15934,2021-04-30,a new model of trickle down sustainability pre...,on thursday investors sent a strong message to...


In [8]:
# конкатенація заголовку та основного змісту новини
df_proc['News'] = df_proc['Heading'] + ' ' + df_proc['Standfirst']
df_proc

,Date,Heading,Standfirst,News
0,2010-02-25,bright future,an investment from goldman sachs the us bank h...,bright future an investment from goldman sachs...
1,2010-02-25,decision makers,he sits on the board of goldman sachs is a mem...,decision makers he sits on the board of goldma...
2,2010-02-25,financial crisis panel to recall bank chiefs,sachs had been creating and selling securities...,financial crisis panel to recall bank chiefs s...
3,2010-02-25,dimon days,sachs and morgan stanley,dimon days sachs and morgan stanley
4,2010-02-25,us stocks fall on greece debt fears,nike the sports wear company rose 19 per cent ...,us stocks fall on greece debt fears nike the s...
...,...,...,...,...
15931,2021-04-29,margin debt and leverage are flashing red again,as john waldron chief operating officer of gol...,margin debt and leverage are flashing red agai...
15932,2021-04-29,goldman sachs predicts quantum computing 5 yea...,,goldman sachs predicts quantum computing 5 yea...
15933,2021-04-29,no media on verizons horizon,,no media on verizons horizon
15934,2021-04-30,a new model of trickle down sustainability pre...,on thursday investors sent a strong message to...,a new model of trickle down sustainability pre...


#### Group text by date

In [9]:
df_proc_group = df_proc.copy()
df_proc_group = df_proc_group.groupby(['Date'])['News'].apply(' '.join).reset_index()
df_proc_group 

,Date,News
0,2010-02-25,bright future an investment from goldman sachs...
1,2010-02-26,investors sceptical on aig prospects earlier t...
2,2010-02-27,73m bofa package for lewis tom montag the gold...
3,2010-02-28,chiles economic tremors unlikely to last alber...
4,2010-03-01,ma burst drives wall street higher riskmetrics...
...,...,...
3461,2021-04-26,pepco sets out store in europe goldman sachs a...
3462,2021-04-27,total bank losses from archegos implosion exce...
3463,2021-04-28,global ipos begin 2021 at breakneck pace a key...
3464,2021-04-29,hollywood mogul ari emanuels endeavor makes vo...


In [10]:
# df_proc_group.set_index('Date', inplace=True)

### finBERT classification

In [67]:
# токенізатор base-uncased
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [68]:
# модель для класифікації
model = BertForSequenceClassification.from_pretrained(r'C:\Users\adima\Desktop\thesis\models\sentiment\finbert\pytorch_model.bin', \
                                                      config=r'C:\Users\adima\Desktop\thesis\models\sentiment\finbert\config.json', \
                                                     num_labels = 3)

In [89]:
# new = df_proc['News'].iloc[5]
# new

'french shipping line misses out on aid jeanyves schapiro also told the marine money conference in hamburg that efforts to bring in private equity investors including goldman sachs and texas pacific group had failed'

In [90]:
# inputs = tokenizer(new, return_tensors="pt")

# outputs = model(**inputs)
# outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.8317,  2.0532, -0.8697]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [91]:
# label_list=['positive','negative','neutral']

In [92]:
# label_list[torch.argmax(outputs[0])]

'negative'

In [215]:
# float(torch.max(outputs[0]))

2.0531585216522217

In [213]:
label_list=['positive','negative','neutral']

# класифікація новин 
def get_sentiment(new):
    
    inputs = tokenizer(new, return_tensors="pt", truncation = True, max_length =512)
    outputs = model(**inputs)
    
    maxlogit = float(torch.max(outputs[0]))
    label = label_list[torch.argmax(outputs[0])]
    
    
    return maxlogit, label

In [214]:
%%time
sentiment_tuple = df_proc_group['News'].apply(get_sentiment)

Wall time: 42min 4s


In [216]:
sent_df = pd.DataFrame(sentiment_tuple.tolist(), columns=['maxlogit', 'label'])
sent_df

,maxlogit,label
0,2.080617,negative
1,1.810755,neutral
2,2.414358,neutral
3,1.556747,neutral
4,0.779622,positive
...,...,...
3461,2.837066,neutral
3462,2.294212,negative
3463,1.310651,negative
3464,0.846088,negative


In [222]:
df_sent_classified = pd.concat([df_proc_group, sent_df], axis=1)
df_sent_classified.set_index('Date', inplace=True)

In [223]:
df_sent_classified

,News,maxlogit,label
Date,,,
2010-02-25,bright future an investment from goldman sachs...,2.080617,negative
2010-02-26,investors sceptical on aig prospects earlier t...,1.810755,neutral
2010-02-27,73m bofa package for lewis tom montag the gold...,2.414358,neutral
2010-02-28,chiles economic tremors unlikely to last alber...,1.556747,neutral
2010-03-01,ma burst drives wall street higher riskmetrics...,0.779622,positive
...,...,...,...
2021-04-26,pepco sets out store in europe goldman sachs a...,2.837066,neutral
2021-04-27,total bank losses from archegos implosion exce...,2.294212,negative
2021-04-28,global ipos begin 2021 at breakneck pace a key...,1.310651,negative


In [225]:
# df_sent_classified.to_csv(r'C:\Users\adima\Desktop\thesis\data_overview\data\pre_ready_data\sentiment_classified_news.csv')